# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [6]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
18,HOBBIES_1_019_CA_1_validation,HOBBIES_1_019,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,31,30,0,5,0,11,4,6,12
31,HOBBIES_1_032_CA_1_validation,HOBBIES_1_032,HOBBIES_1,HOBBIES,CA_1,CA,9,0,5,16,...,7,4,5,3,6,0,0,2,4,4
35,HOBBIES_1_036_CA_1_validation,HOBBIES_1_036,HOBBIES_1,HOBBIES,CA_1,CA,2,1,0,0,...,3,4,0,0,1,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691,HOBBIES_2_129_CA_4_validation,HOBBIES_2_129,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9694,HOBBIES_2_132_CA_4_validation,HOBBIES_2_132,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9703,HOBBIES_2_141_CA_4_validation,HOBBIES_2_141,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9704,HOBBIES_2_142_CA_4_validation,HOBBIES_2_142,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [7]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [8]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---


In [9]:
data_pattern = data_cleaning(sales_lumpy)

In [10]:
data_pattern

,Date,HOBBIES_1_006_CA_1,HOBBIES_1_006_CA_2,HOBBIES_1_008_CA_2,HOBBIES_1_008_CA_4,HOBBIES_1_009_CA_1,HOBBIES_1_009_CA_2,HOBBIES_1_009_CA_3,HOBBIES_1_009_CA_4,HOBBIES_1_015_CA_2,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,0.0,0.0,0.0,2.0,2.0,3.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,18.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,1.0,0.0,2.0,23.0,0.0,0.0,0.0,2.0,7.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,1.0,0.0,11.0,9.0,0.0,3.0,0.0,1.0,7.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,0.0,0.0,5.0,8.0,2.0,8.0,1.0,4.0,22.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,0.0,0.0,36.0,5.0,0.0,0.0,2.0,0.0,28.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,0.0,0.0,8.0,6.0,0.0,1.0,2.0,2.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,0.0,0.0,4.0,34.0,16.0,0.0,0.0,0.0,6.0,...,0,0,0,1,0,0,0,1,0,0


# XGBoost Hyperparameters tuning to find optimum parameters

In [11]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [12]:
end_train_date = '2016-04-10'

In [13]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [14]:
correlation_threshold = 0.8

In [15]:
# User-defined funtion to find optimum parameters on training data
def XGBoost_Hyperparameters_Tuning(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        params = { 'max_depth': [3, 5, 7, 10],
                  'learning_rate': [0.01, 0.1, 0.2, 0.3],
                  'subsample': np.arange(0.5, 1.0, 0.1),
                  'colsample_bytree': np.arange(0.5, 1.0, 0.1),
                  'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
                  'n_estimators': [30, 50, 100]}

        xgbr = xgb.XGBRegressor(seed = 20)

        regressor = RandomizedSearchCV(estimator=xgbr,
                                      param_distributions=params,
                                      scoring='neg_mean_squared_error',
                                      n_iter=100,
                                      verbose=1)
        regressor.fit(X_train, y_train)
        best_params_temp = regressor.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
list_pattern_sample = sample(list_lumpy, 30)

In [ ]:
df_output_hyperparams_tuning = XGBoost_Hyperparameters_Tuning(list_pattern_sample, not_id_cols, lags, moving_averages,
                                                              stds, maximums, minimums, totals, num_days_zeros,
                                                              correlation_threshold, end_train_date)

Streaming output truncated to the last 5000 lines.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:48:21] WARNING: /workspace/src/objective/regression_obj.

In [ ]:
df_output_hyperparams_tuning

,subsample,n_estimators,max_depth,learning_rate,colsample_bytree,colsample_bylevel,Product
0,0.8,30.0,5.0,0.10,0.6,0.6,FOODS_3_029_CA_3
1,0.8,30.0,3.0,0.10,0.6,0.5,FOODS_3_427_CA_4
2,0.7,100.0,7.0,0.01,0.9,0.9,FOODS_3_354_CA_1
3,0.6,50.0,3.0,0.10,0.5,0.7,FOODS_3_214_CA_4
4,0.9,30.0,3.0,0.10,0.6,0.6,FOODS_1_135_CA_2
5,0.6,100.0,10.0,0.01,0.8,0.7,FOODS_3_273_CA_2
6,0.5,100.0,3.0,0.01,0.6,0.9,FOODS_3_721_CA_4
7,0.7,100.0,3.0,0.01,0.8,0.9,FOODS_1_136_CA_3
8,0.8,30.0,3.0,0.20,0.8,0.5,FOODS_3_565_CA_2
9,0.5,50.0,3.0,0.10,0.5,0.5,FOODS_3_541_CA_1


In [ ]:
df_output_hyperparams_tuning.to_csv('Lumpy_XGBoost_Hyperparameter_Tuning.csv')

# Decide optimum set of parameters for XGBoost algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyperparams_tuning.groupby(['subsample', 'n_estimators', 'max_depth', 
                         'learning_rate', 'colsample_bytree',
                         'colsample_bylevel'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
subsample n_estimators max_depth learning_rate colsample_bytree colsample_bylevel         
0.8       30.0         3.0       0.10          0.6              0.5                      3
0.5       30.0         3.0       0.10          0.9              0.9                      1
          50.0         3.0       0.10          0.5              0.5                      1
0.9       30.0         3.0       0.10          0.6              0.7                      1
                                                                0.6                      1
0.8       50.0         3.0       0.01          0.5              0.7                      1
          30.0         5.0       0.10          0.6              0.6                      1
                       3.0       0.20          0.8              0.5                      1
                                 0.01          0.9              0.7                      1
0.7       100.0        7.0       0.01          0.9              0.9                      1
                                               0.6              0.5                      1
                       3.0       0.01          0.8              0.9                      1
          50.0         3.0       0.10          0.8              0.5                      1
                                               0.5              0.6                      1
          30.0         5.0       0.10          0.8              0.6                      1
0.6       100.0        10.0      0.01          0.8              0.7                      1
                       7.0       0.01          0.5              0.5                      1
          50.0         5.0       0.10          0.5              0.7                      1
                       3.0       0.10          0.5              0.7                      1
          30.0         10.0      0.01          0.5              0.8                      1
                       3.0       0.10          0.8              0.5                      1
                                               0.7              0.5                      1
                                               0.6              0.6                      1
0.5       100.0        3.0       0.10          0.6              0.8                      1
                                 0.01          0.6              0.9                      1
                                               0.5              0.9                      1
          50.0         3.0       0.10          0.6              0.5                      1
0.9       100.0        5.0       0.01          0.6              0.5                      1

In [ ]:
# See what value of parameter "subsample" occurs the most frequently
df_output_hyperparams_tuning.groupby(['subsample'])[['Product']].count()

,Product
subsample,
0.5,6
0.6,8
0.7,6
0.8,7
0.9,3


In [ ]:
# See what value of parameter "n_estimators" occurs the most frequently
df_output_hyperparams_tuning.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30.0,14
50.0,7
100.0,9


In [ ]:
# See what value of parameter "max_depth" occurs the most frequently
df_output_hyperparams_tuning.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3.0,21
5.0,4
7.0,3
10.0,2


In [ ]:
# See what value of parameter "learning_rate" occurs the most frequently
df_output_hyperparams_tuning.groupby(['learning_rate'])[['Product']].count()

,Product
learning_rate,
0.01,11
0.10,18
0.20,1


In [ ]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyperparams_tuning.groupby(['colsample_bytree'])[['Product']].count()

,Product
colsample_bytree,
0.5,8
0.6,12
0.7,1
0.8,6
0.9,3


In [ ]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyperparams_tuning.groupby(['colsample_bylevel'])[['Product']].count()

,Product
colsample_bylevel,
0.5,12
0.6,5
0.7,6
0.8,2
0.9,5


# Implement XGBoost on Test Set with optimum set of parameters

**Specify optimum set of parameters**

In [16]:
subsample_chosen = 0.8
n_estimators_chosen = 30
max_depth_chosen = 3
learning_rate_chosen = 0.1               
colsample_bytree_chosen =  0.6
colsample_bylevel_chosen = 0.5

In [17]:
# User-defined function to run XGBoost on Test Data

def XGBoost_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                      maximums, minimums, totals, num_days_zeros, correlation_threshold,
                      end_train_date, subsample_chosen, n_estimators_chosen,
                      max_depth_chosen, learning_rate_chosen, 
                      colsample_bytree_chosen, colsample_bylevel_chosen):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_xgb_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        xgb_r = xgb.XGBRegressor(seed = 20,
                                subsample = subsample_chosen,
                                n_estimators = n_estimators_chosen,
                                max_depth = max_depth_chosen,
                                learning_rate = learning_rate_chosen,               
                                colsample_bytree = colsample_bytree_chosen,
                                colsample_bylevel = colsample_bylevel_chosen)
        

        xgb_r.fit(X_train,y_train)
        y_pred = xgb_r.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_xgb_pattern = df_output_xgb_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_xgb_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [18]:
# Start running XGBoost on Test Set

start = timer()

df_forecast_values = XGBoost_Test_Data(list_lumpy, not_id_cols, lags, moving_averages, stds,
                                      maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                      end_train_date, subsample_chosen, n_estimators_chosen,
                                      max_depth_chosen, learning_rate_chosen, 
                                      colsample_bytree_chosen, colsample_bylevel_chosen)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_006_CA_1
Progressing: 0.0 %
[16:37:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOBBIES_1_006_CA_2
Progressing: 0.29 %
[16:37:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOBBIES_1_008_CA_2
Progressing: 0.58 %
[16:37:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOBBIES_1_008_CA_4
Progressing: 0.87 %
[16:37:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOBBIES_1_009_CA_1
Progressing: 1.16 %
[16:37:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOBBIES_1

In [19]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,3.0,0.859721,HOBBIES_1_006_CA_1
2016-04-12,0.0,0.934120,HOBBIES_1_006_CA_1
2016-04-13,0.0,1.090522,HOBBIES_1_006_CA_1
2016-04-14,0.0,0.504981,HOBBIES_1_006_CA_1
2016-04-15,2.0,0.865468,HOBBIES_1_006_CA_1
...,...,...,...
2016-04-20,0.0,0.349071,HOBBIES_2_149_CA_3
2016-04-21,0.0,0.235814,HOBBIES_2_149_CA_3
2016-04-22,0.0,0.266282,HOBBIES_2_149_CA_3


In [20]:
df_forecast_values.isnull().sum()

Actual Data    0
Forecast       0
Product        0
dtype: int64

In [21]:
df_forecast_values.to_csv('XGBoost_Lumpy_Forecast_Values.csv')

In [22]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [23]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.03805539225000227 minutes


In [24]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_006_CA_1,1.063671,148.9140,169.4603,62.9862,1.063671,1.168538
1,HOBBIES_1_006_CA_2,1.347640,217.6957,165.6838,29.5403,0.621988,0.681878
2,HOBBIES_1_008_CA_2,0.679748,62.4217,73.7661,48.0610,5.751710,9.379254
3,HOBBIES_1_008_CA_4,0.580327,68.6901,73.5035,106.6003,5.446141,8.871673
4,HOBBIES_1_009_CA_1,0.515224,102.5644,137.5396,48.8015,2.417589,4.596841
...,...,...,...,...,...,...,...
340,HOBBIES_2_148_CA_2,3.151127,339.3521,195.2875,80.2485,0.242394,0.288531
341,HOBBIES_2_148_CA_3,1.550565,333.9678,195.7537,82.5436,0.238548,0.290781
342,HOBBIES_2_149_CA_1,0.722980,155.7188,197.2909,94.9967,0.333683,0.593922
343,HOBBIES_2_149_CA_2,inf,inf,200.0000,NaN,0.132341,0.145648


In [25]:
df_forecast_metrics.to_csv('XGBoost_Lumpy_Forecast_Metrics.csv')